<a href="https://colab.research.google.com/github/vondervish/Netflix_Recommendation_system/blob/main/Netflix_recommendation_sytem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')
    basepath = '/content/drive/MyDrive/Projects/'
else:
    basepath = '/C:/Users/Vishu/Downloads/Projects/Netflix'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from pathlib import Path

In [3]:
base_folder = Path(basepath)
data_folder = base_folder/'Data/Netflix'
model_folder = base_folder/'Models/Netflix'

In [4]:
model_folder.mkdir(exist_ok=True, parents = True)

In [5]:
model_folder

PosixPath('/content/drive/MyDrive/Projects/Models/Netflix')

In [6]:
dataset = data_folder/'Cleaned_netflix.csv'

In [7]:
dataset

PosixPath('/content/drive/MyDrive/Projects/Data/Netflix/Cleaned_netflix.csv')

In [8]:
!pip install sentence-transformers

In [9]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [10]:
netflix_data = pd.read_csv(dataset)

In [11]:
netflix_data

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,added_month,added_year
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,"September 25, 2021",2020,13+,90,Documentaries,"As her father nears the end of his life, filmm...",9,2021
1,s2,TV Show,Blood & Water,Unknown,Ama Qamata,South Africa,"September 24, 2021",2021,17+,1800,International TV Shows,"After crossing paths at a party, a Cape Town t...",9,2021
2,s2,TV Show,Blood & Water,Unknown,Khosi Ngema,South Africa,"September 24, 2021",2021,17+,1800,International TV Shows,"After crossing paths at a party, a Cape Town t...",9,2021
3,s2,TV Show,Blood & Water,Unknown,Gail Mabalane,South Africa,"September 24, 2021",2021,17+,1800,International TV Shows,"After crossing paths at a party, a Cape Town t...",9,2021
4,s2,TV Show,Blood & Water,Unknown,Thabang Molaba,South Africa,"September 24, 2021",2021,17+,1800,International TV Shows,"After crossing paths at a party, a Cape Town t...",9,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201986,s8807,Movie,Zubaan,Mozez Singh,Manish Chaudhary,India,"March 2, 2019",2015,14+,111,Music & Musicals,A scrappy but poor boy worms his way into a ty...,3,2019
201987,s8807,Movie,Zubaan,Mozez Singh,Meghna Malik,India,"March 2, 2019",2015,14+,111,Music & Musicals,A scrappy but poor boy worms his way into a ty...,3,2019
201988,s8807,Movie,Zubaan,Mozez Singh,Malkeet Rauni,India,"March 2, 2019",2015,14+,111,Music & Musicals,A scrappy but poor boy worms his way into a ty...,3,2019
201989,s8807,Movie,Zubaan,Mozez Singh,Anita Shabdish,India,"March 2, 2019",2015,14+,111,Music & Musicals,A scrappy but poor boy worms his way into a ty...,3,2019


In [30]:
# Step 1: Load Data
data = netflix_data[['title', 'description', 'type', 'rating']].dropna()  # Keep relevant columns
data['description'] = data['description'].str.lower()  # Convert to lowercase

In [31]:
# Remove duplicates based on title and description
data = data.drop_duplicates(subset=['title', 'description'])

In [32]:
data

,title,description,type,rating
0,Dick Johnson Is Dead,"as her father nears the end of his life, filmm...",Movie,13+
1,Blood & Water,"after crossing paths at a party, a cape town t...",TV Show,17+
58,Ganglands,to protect his family from a powerful drug lor...,TV Show,17+
85,Jailbirds New Orleans,"feuds, flirtations and toilet talk go down amo...",TV Show,17+
87,Kota Factory,in a city of coaching centers known to train i...,TV Show,17+
...,...,...,...,...
201902,Zodiac,"a political cartoonist, a crime reporter and a...",Movie,17+
201932,Zombie Dumb,"while living alone in a spooky town, a young g...",TV Show,7+
201935,Zombieland,looking to survive in a world taken over by zo...,Movie,17+
201949,Zoom,"dragged from civilian life, a former superhero...",Movie,Parental guidance


In [33]:
# Step 2: Load Pre-Trained Sentence Transformer Model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight model for semantic similarity

In [34]:
data = data.reset_index(drop=True)  # Ensure indices are continuous
data['embeddings'] = list(model.encode(data['description'], show_progress_bar=True))

Batches:   0%|          | 0/276 [00:00<?, ?it/s]

In [35]:
# Step 4: Compute Pairwise Similarity
embeddings = list(data['embeddings'])
cosine_sim = cosine_similarity(embeddings, embeddings)

In [36]:
def recommend(title, cosine_sim=cosine_sim, data=data, top_n=5, filters=None):
    """
    Recommend similar shows/movies based on the description.

    Parameters:
        title (str): Title of the show/movie for which recommendations are needed.
        cosine_sim (ndarray): Pairwise cosine similarity matrix.
        data (DataFrame): DataFrame containing titles, descriptions, and optional filters.
        top_n (int): Number of recommendations to return.
        filters (dict): Optional filters to apply. Example: {"type": "Movie", "rating": "13+"}.

    Returns:
        list: Recommended titles that match the filters.
    """
    # Find the index of the title
    idx = data.index[data['title'] == title].tolist()
    if not idx:
        return f"Title '{title}' not found in the dataset."

    idx = idx[0]  # Get the first match

    # Get similarity scores for all items
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort by similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get top N recommendations (excluding the input)
    sim_scores = sim_scores[1:]  # Exclude the input title
    recommended_indices = [i[0] for i in sim_scores]

    # Apply filters if specified
    if filters:
        filtered_indices = []
        for i in recommended_indices:
            match = True
            for key, value in filters.items():
                if key in data.columns and data.iloc[i][key] != value:
                    match = False
                    break
            if match:
                filtered_indices.append(i)
        recommended_indices = filtered_indices

    # Limit to top N recommendations
    recommended_indices = recommended_indices[:top_n]

    # Return titles of recommended items
    recommended_titles = [data['title'].iloc[i] for i in recommended_indices]
    return recommended_titles

In [37]:
# Example Usage with Filters
filters = {"type": "Movie", "rating": "13+"}
recommendations = recommend("Blood & Water", filters=filters)
print("Recommended Titles:", recommendations)

Recommended Titles: ['Rumor Has It', 'Gimme Shelter', 'The Space Between Us', 'Barely Lethal', 'Wild Oats']


In [38]:
filters = {"type": "TV Show", "rating": "17+"}
recommendations = recommend("Blood & Water", filters=filters)
print("Recommended TV Shows:", recommendations)

Recommended TV Shows: ['High Seas', 'Teenage Bounty Hunters', 'Warrior Nun', 'Vampires', 'Trinkets']


In [43]:
recommendations = recommend("Blood & Water")
print("Recommended Titles:", recommendations)

Recommended Titles: ['Rocks', 'Dive Club', 'Jem and the Holograms', 'Scales: Mermaids Are Real', 'Night Comes On']


In [44]:
import os
import pickle

# Step 1: Create the model folder if it doesn't exist
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

# Step 2: Save the Sentence Transformer model
model.save(os.path.join(model_folder, "sentence_transformer_model"))

# Step 3: Save the embeddings and cosine similarity matrix
with open(os.path.join(model_folder, "embeddings.pkl"), "wb") as f:
    pickle.dump(embeddings, f)

with open(os.path.join(model_folder, "cosine_similarity.pkl"), "wb") as f:
    pickle.dump(cosine_sim, f)

print("Model, embeddings, and cosine similarity matrix saved successfully.")

Model, embeddings, and cosine similarity matrix saved successfully.
